In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split

import os, random

In [2]:
path = os.getcwd()

data_dir = str(path)+"\input"
train_dir = str(path)+"\input\\train"
test_dir = str(path)+"\input\\test"

In [3]:
labels = pd.read_csv("train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [4]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


In [5]:
class CactiImageData(Dataset):
    def __init__(self, split_data, data_root = './', transform=None):
        super().__init__()
        self.df = split_data.values
        self.data_root = data_root
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_root, img_name)
        image = mpimg.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [6]:
#prepare training and validation data
batch_size = 128

train, valid = train_test_split(labels, stratify=labels.has_cactus, test_size=0.2)

transformations = transforms.Compose([transforms.ToPILImage(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])

train_data = CactiImageData(split_data = train, data_root = train_dir, transform = transformations)
valid_data = CactiImageData(split_data = valid, data_root = train_dir, transform = transformations)

train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True, num_workers = 0)
valid_loader = DataLoader(dataset = valid_data, batch_size = batch_size//2, shuffle = False, num_workers = 0)

In [7]:
#prepare testing data
sample_submission = pd.read_csv('sample_submission.csv')
test_data = CactiImageData(split_data = sample_submission, data_root = test_dir, transform = transformations)
test_loader = DataLoader(dataset = test_data, batch_size=32, shuffle=False, num_workers=0)

In [8]:
#define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
                nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=1,padding=0),nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(
                nn.Conv2d(in_channels=32,out_channels=64,kernel_size=2,stride=1,padding=1),nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),nn.MaxPool2d(2,2))
        self.conv4 = nn.Sequential(
                nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1),nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),nn.MaxPool2d(2,2))
        self.fc = nn.Sequential(nn.Linear(256 * 2 * 2, 1024),nn.ReLU(inplace=True),nn.Dropout(0.25),nn.Linear(1024, 2))

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.shape[0],-1)
        x = self.fc(x)
        return x

In [9]:
# define our loss function and optimizer
model = Net()
if train_on_gpu:
    model.cuda()
print(model)    

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNo

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [11]:
# number of epochs to train the model
n_epochs = 40

valid_loss_min = np.Inf # track change in validation loss
path = os.getcwd()
path = str(path)+"\model_cactus.pt"

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for images, labels in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate the batch loss
        loss = criterion(output, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*images.size(0)
        
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for images, labels in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)
        # calculate the batch loss
        loss = criterion(output, labels)
        # update average validation loss 
        valid_loss += loss.item()*images.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
    
        torch.save(model.state_dict(), path)
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.147685 	Validation Loss: 0.060431
Validation loss decreased (inf --> 0.060431).  Saving model ...
Epoch: 2 	Training Loss: 0.049857 	Validation Loss: 0.099850
Epoch: 3 	Training Loss: 0.040630 	Validation Loss: 0.050686
Validation loss decreased (0.060431 --> 0.050686).  Saving model ...
Epoch: 4 	Training Loss: 0.026800 	Validation Loss: 0.031981
Validation loss decreased (0.050686 --> 0.031981).  Saving model ...
Epoch: 5 	Training Loss: 0.023919 	Validation Loss: 0.022926
Validation loss decreased (0.031981 --> 0.022926).  Saving model ...
Epoch: 6 	Training Loss: 0.019426 	Validation Loss: 0.033306
Epoch: 7 	Training Loss: 0.011917 	Validation Loss: 0.033852
Epoch: 8 	Training Loss: 0.012055 	Validation Loss: 0.027585
Epoch: 9 	Training Loss: 0.012459 	Validation Loss: 0.043890
Epoch: 10 	Training Loss: 0.013289 	Validation Loss: 0.077263
Epoch: 11 	Training Loss: 0.006952 	Validation Loss: 0.065658
Epoch: 12 	Training Loss: 0.007304 	Validation Loss: 0.0

KeyboardInterrupt: 

In [12]:
#Load the Model with the Lowest Validation Loss
model.load_state_dict(torch.load('model_cactus.pt'))

In [15]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))


model.eval()
# iterate over test data
for images, labels in train_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(images)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update test loss 
    test_loss += loss.item()*images.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
   # for i in range(batch_size):
    for i in range(len(labels)):
        label = labels.data[i]
        if(pred[i] == label):
           class_correct[label] += 1
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(train_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


Test Loss: 0.008305


Test Accuracy (Overall): 99% (13969/14000)
